### SAR-FLOOD MAPPING USING A CHANGE DETECTION APPROACH
  ===========================================================================================
  Within this script SAR Sentinel-1 is being used to generate a flood extent map. A change 
  detection approach was chosen, where a before- and after-flood event image will be compared. 
  Sentinel-1 GRD imagery is being used. Ground Range Detected imagery includes the following 
  preprocessing steps: Thermal-Noise Removal, Radiometric calibration, Terrain-correction 
  hence only a Speckle filter needs to be applied in the preprocessing.  
  https://un-spider.org/advisory-support/recommended-practices/recommended-practice-google-earth-engine-flood-mapping/step-by-step#Step%202:%20Time%20frame%20and%20sensor%20parameters%20selection


In [81]:
print('hello world')

hello world


In [4]:
import ee
ee.Initialize()

In [43]:
# Get area of interest 'Mozambique'
country = ee.FeatureCollection("FAO/GAUL/2015/level0").filterMetadata('ADM0_NAME', 'equals', 'Mozambique')

In [58]:
# Window for images before flooding
before_start= '2020-05-01'
before_end='2020-10-15'

# Dates for flooding period
after_start='2020-11-04'
after_end='2020-11-25'

# Parameters for SAR
polarization = "VH"
pass_direction = "ASCENDING"
difference_threshold = 1.25
# These orbits need to be change.  
# Are they necessary given country bounds?
relative_orbits = [63, 165, 92]

In [59]:
# is the following necessary if we remove relative orbits?
# for orbit in relative_orbits:
collection = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", polarization))
    .filter(ee.Filter.eq("orbitProperties_pass", pass_direction))
    .filter(ee.Filter.eq("resolution_meters", 10))
    #.filter(ee.Filter.eq("relativeOrbitNumber_start", orbit))
    .filterBounds(country)
    .select(polarization)
)

# Select images by predefined dates and get the median values of the 'before' image
before_collection = collection.filterDate(before_start, before_end)
before_med = ee.Image(before_collection.reduce(ee.Reducer.median()))
after_collection = collection.filterDate(after_start, after_end)


In [60]:
geom = after_collection.geometry()
#Map.addLayer(geom, {}, 'Imagery extent ' + orbit.toString())


In [ ]:
# Extract date from meta data
def dates(imgcol):
    range = imgcol.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
    printed = (
        ee.String('from ')
        .cat(ee.Date(range.get('min')).format('YYYY-MM-dd'))
        .cat(' to ')
        .cat(ee.Date(range.get('max')).format('YYYY-MM-dd'))
    )
    return printed

# print dates of before images to console
before_count = before_collection.size()
print(ee.String('Tiles selected: Before Flood ({})'.format(before_count)))
print(dates(before_collection))
print(before_collection)

# print dates of after images to console
after_count = before_collection.size()
print(ee.String('Tiles selected: After Flood  ({})'.format(after_count)))
print(dates(after_collection))
print(after_collection)

# Create a mosaic of selected tiles and clip to study area
before = before_med.clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing  
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

# # Export the image, specifying scale and region.
# Export.image.toDrive({
#   image: after_filtered,
#   description: 'flood_imagery_',
#   maxPixels: 122137795
# })


#------------------------------- FLOOD EXTENT CALCULATION -------------------------------//

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask 
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine flood result using additional datasets
      
# Include JRC layer on surface water seasonality to mask flood pixels from areas
# of "permanent" water (where there is water > 10 months of the year)
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))

#Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
flooded_mask = difference_binary.where(swater_mask,0)
# final flooded area without pixels in perennial waterbodies
flooded = flooded_mask.updateMask(flooded_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the flood extent product 
connections = flooded.connectedPixelCount()    
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 10 percent slope using a Digital Elevation Model 
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(10))

# Before and after flood SAR mosaic
Map.centerObject(aoi,8)
Map.addLayer(before_filtered, {min:-25,max:0}, 'Before Flood ',0)
Map.addLayer(after_filtered, {min:-25,max:0}, 'After Flood ',1)

# Difference layer
Map.addLayer(difference,{min:0,max:2},"Difference Layer ",0)

# Flooded areas
Map.addLayer(flooded,{palette:"0000FF"},'Flooded areas ' )


#------------------------------------- EXPORTS ------------------------------------#
# Export flooded area as TIFF file 
Export.image.toDrive({
  image: flooded, 
  description: 'Flood_extent_raster_',
  fileNamePrefix: 'flooded_' ,
  region: aoi, 
  maxPixels: 1e10
})

# Export flooded area as shapefile (for further analysis in e.g. QGIS)
# Convert flood raster to polygons
flooded_vec = flooded.reduceToVectors({
  scale: 10,
  geometryType:'polygon',
  geometry: aoi,
  eightConnected: false,
  bestEffort:true,
  tileScale:2,
})

# Export flood polygons as shape-file
Export.table.toDrive({
  collection:flooded_vec,
  description:'Flood_extent_vector_' ,
  fileFormat:'SHP',
  fileNamePrefix:'flooded_vec_' 
})

In [8]:
country.draw()

In [10]:
empty = ee.Image().byte()
outline = empty.paint(country, 'red', 2)

In [20]:
from IPython.display import Image

Image(url = outline.getThumbUrl({'min': 0, 'max': 1000, 'dimensions': 812,
                'palette': 'red'}))

In [75]:
### Set up Folium for interactive mapping ###

# Modified from GitHub notebook here: 
# Import the folium library.
import folium
from folium import plugins

# Add custom basemaps to folium
basemaps = {
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [80]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['red', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[-30, 30], zoom_start=6)

# Add custom basemaps
basemaps['Google Terrain'].add_to(my_map)
basemaps['Google Satellite'].add_to(my_map)

# Add the elevation model to the map object.
country.vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['red', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

my_map.add_ee_layer(country, country.vis_params, 'Mozambique')

my_map.add_ee_layer(geom, {'min': 0,
  'max': 4000}, 'Satellite extent')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())


# Display the map.
display(my_map)


# Steps to do:

- Right dates
- Difference (binary)
- Speckle remove
- Mask out permanent water
- Population - mask from water
- summary stats on population